In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import necessary Libraries

In [5]:
import warnings
warnings.filterwarnings('ignore')

import os
import re
import random
from collections import Counter
from numpy import where


import networkx as nx
from matplotlib import pyplot
import matplotlib.pyplot as plt
%matplotlib inline
print('NetworkX version: {}'.format(nx.__version__))
import networkx.algorithms.community as nx_comm


import pandas as pd
from numpy import mean
import numpy as np

NetworkX version: 3.1


## Function to count the number of Triangles(Cycles of length 3)

In [6]:
def find_triangles(A):
  cycle3_1 = []
  for i in range(0,A.shape[0]):
    for j in range(0,A.shape[0]):
      if(A[i][j]>0): #if edge exists
        for k in range(0,A.shape[0]):
          if(i!=k and A[j][k]>0 and A[k][i]>0):  #if other two edges exist
            if([i,j,k] not in cycle3_1 and [j,i,k] not in cycle3_1 and [k,i,j] not in cycle3_1 and [i,k,j] not in cycle3_1 and [j,k,i] not in cycle3_1 and [k,j,i] not in cycle3_1):
              cycle3_1.append([i,j,k])
              #print(characters[i],characters[j],characters[k]);
  return cycle3_1
          

## Function to count the number of Hexagons(Cycles of length 6 with timeout)

In [7]:
import signal
import time
def timeout_handler(signum, frame):
    raise TimeoutError("Function execution time exceeded 10 minutes.")

In [8]:
def find_hexagons(A):
  signal.signal(signal.SIGALRM, timeout_handler)
  # Set the alarm for 5 minutes
  signal.alarm(300)
  try:
    cycle6_1 = []

    for i in range(0,A.shape[0]):
      for j in range(0,A.shape[0]):
        if(A[i][j]>0): #edge btw i and j
          for k in range(0,A.shape[0]):
            if(i!=k and A[j][k]>0): #edge btw j and k
              for l in range(0,A.shape[0]):
                if(i!=l and j!=l and A[k][l]>0): #edge between l and k
                  for m in range(0,A.shape[0]):
                    if(i!=m and j!=m and k!=m and A[l][m]>0): #edge between l and m
                      for n in range(0,A.shape[0]):
                        if(i!=n and j!=n and k!=n and l!=n and A[m][n]>0 and A[n][i]>0): #edge between m and n and edge btw n and i to complete the cycle
                          whatever = []
                          whatever.append(i)
                          whatever.append(j)
                          whatever.append(k)
                          whatever.append(l)
                          whatever.append(m)
                          whatever.append(n)
                          whatever.sort()
                          if(whatever not in cycle6_1):
                            cycle6_1.append(whatever)

                        # print(characters[i],characters[j],characters[k],characters[l],characters[m],characters[n])
    return cycle6_1  
  except TimeoutError:
    return 0
  finally:
    signal.alarm(0)

In [9]:
!pip install torch_geometric
from torch_geometric.data import Data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=f1156669661e444738f3eceedaf8e9e485df0e1a191174154671cdbad0c439d5
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


## Creating First Part of the Dataset using 5G Conspiracy Graph

In [150]:
datasets = []
count = 0

In [151]:
import os
directory='/content/drive/MyDrive/Chaitraa/5G_Conspiracy_Graphs'
i=0
edge_path = 'edges.txt'
node_path = 'nodes.csv'
for subdir in os.listdir(directory):
  count = count + 1
  if(subdir!=".DS_Store"):
    fh = open(os.path.join(directory,subdir,edge_path), "r")

    print(subdir)
    G = nx.read_edgelist(fh, nodetype=int)
    if(len(G)!=0):
      A = nx.adjacency_matrix(G)
      A = A.todense()
      #print(find_triangles(A))
      fh.close()

      node_attr = pd.read_csv(os.path.join(directory,subdir,node_path))
      graph_node_list=G.nodes()
      node_with_attributes= node_attr['id'].tolist()
      node_attr = node_attr.assign(triangle = 0)
      node_attr = node_attr.assign(hexagon = 0)
      for n in node_with_attributes:
        G.add_node(n)
      node_list =node_with_attributes 
      for index, row in node_attr.iterrows():
        if row['id'] in node_list:
          G.nodes[row['id']]['time'] = row['time']
          G.nodes[row['id']]['friends'] = row['friends']
          G.nodes[row['id']]['followers'] = row['followers']
      #print(A)
      X = np.zeros((A.shape[0], 1))
      tri = find_triangles(A)
      #print(tri)
      for i in tri:
        for node in i:
          X[node][0]+=1
      Y = np.zeros((A.shape[0], 1))
      hex=find_hexagons(A)
      #print(hex)
      if(hex!=0):
        for i in hex:
          for node in i:
            Y[node][0]+=1
      for i in range(0,len(Y)):
        node_attr['triangle'][i] = int(X[i][0])
        node_attr['hexagon'][i] = int(Y[i][0])
      x = node_attr.to_numpy()
      x=torch.from_numpy(x)
      y=np.array([0])
      y=torch.from_numpy(y) 
      adj_t = torch.tensor(A)
      edge_index = adj_t.nonzero().t().contiguous()
      data = Data(x=x, edge_index=edge_index,y=y)
      datasets.append(data)
      continue
   
print(count)

407
408
409
41
410
411
412
42
43
44
45
46
47
48
49
5
50
51
52
53
54
55
56
57
58
59
6
60
61
62
63
64
65
66
67
68
69
7
70
71
72
73
74
75
76
77
78
79
8
80
81
82
83
84
85
86
87
88
89
9
90
91
92
93
94
95
96
97
98
99
326
327
328
329
33
330
331
332
333
334
335
336
337
338
339
34
340
341
342
343
344
345
346
347
348
349
35
350
351
352
353
354
355
356
357
358
359
36
360
361
362
363
364
365
366
367
368
369
37
370
371
372
373
374
375
376
377
378
379
38
380
381
382
383
384
385
386
387
388
389
39
390
391
392
393
394
395
396
397
398
399
4
40
400
401
402
403
404
405
406
236
237
238
239
24
240
241
242
243
244
245
246
247
248
249
25
250
251
252
253
254
255
256
257
258
259
26
260
261
262
263
264
265
266
267
268
269
27
270
271
272
273
274
275
276
277
278
279
28
280
281
282
283
284
285
286
287
288
289
29
290
291
292
293
294
295
296
297
298
299
3
30
300
301
302
303
304
305
306
307
308
309
31
310
311
312
313
314
315
316
317
318
319
32
320
321
322
323
324
325
191
192
193
194
195
196
197
198
199
2
20
200
201
2

In [152]:
datasets

[Data(x=[93, 6], edge_index=[2, 176], y=[1]),
 Data(x=[95, 6], edge_index=[2, 160], y=[1]),
 Data(x=[24, 6], edge_index=[2, 58], y=[1]),
 Data(x=[9, 6], edge_index=[2, 8], y=[1]),
 Data(x=[61, 6], edge_index=[2, 32], y=[1]),
 Data(x=[21, 6], edge_index=[2, 78], y=[1]),
 Data(x=[18, 6], edge_index=[2, 28], y=[1]),
 Data(x=[70, 6], edge_index=[2, 280], y=[1]),
 Data(x=[52, 6], edge_index=[2, 456], y=[1]),
 Data(x=[5, 6], edge_index=[2, 10], y=[1]),
 Data(x=[97, 6], edge_index=[2, 2080], y=[1]),
 Data(x=[24, 6], edge_index=[2, 62], y=[1]),
 Data(x=[8, 6], edge_index=[2, 12], y=[1]),
 Data(x=[97, 6], edge_index=[2, 228], y=[1]),
 Data(x=[55, 6], edge_index=[2, 230], y=[1]),
 Data(x=[58, 6], edge_index=[2, 116], y=[1]),
 Data(x=[90, 6], edge_index=[2, 120], y=[1]),
 Data(x=[89, 6], edge_index=[2, 52], y=[1]),
 Data(x=[98, 6], edge_index=[2, 288], y=[1]),
 Data(x=[11, 6], edge_index=[2, 20], y=[1]),
 Data(x=[48, 6], edge_index=[2, 528], y=[1]),
 Data(x=[48, 6], edge_index=[2, 50], y=[1]),
 D

In [ ]:
torch.save(datasets,'hexagon_407_0.pt')

## Creating second part of Dataset with Non-Conspiracy Graph

In [13]:
count = 0
datasets1 = []

In [14]:
import torch

In [15]:
import os
directory='/content/drive/MyDrive/Chaitraa/Non_Conspiracy_Graphs'
i=0
edge_path = 'edges.txt'
node_path = 'nodes.csv'
for subdir in os.listdir(directory):
  count = count + 1
  if(subdir!=".DS_Store" and count<420):
    fh = open(os.path.join(directory,subdir,edge_path), "r")

    print(subdir)
    G = nx.read_edgelist(fh, nodetype=int)
    if(len(G)!=0):
      A = nx.adjacency_matrix(G)
      A = A.todense()
      # print(find_triangles(A))
      fh.close()

      node_attr = pd.read_csv(os.path.join(directory,subdir,node_path))
      graph_node_list=G.nodes()
      node_with_attributes= node_attr['id'].tolist()
      node_attr = node_attr.assign(triangle = 0)
      node_attr = node_attr.assign(hexagon = 0)
      for n in node_with_attributes:
        G.add_node(n)
      node_list =node_with_attributes 
      for index, row in node_attr.iterrows():
        if row['id'] in node_list:
          G.nodes[row['id']]['time'] = row['time']
          G.nodes[row['id']]['friends'] = row['friends']
          G.nodes[row['id']]['followers'] = row['followers']
      # print(A)
      X = np.zeros((A.shape[0], 1))
      tri = find_triangles(A)
      #print(tri)
      for i in tri:
        for node in i:
          X[node][0]+=1
      Y = np.zeros((A.shape[0], 1))
      hex=find_hexagons(A)

      if(hex!=0):
        for i in hex:
          for node in i:
            Y[node][0]+=1
      for i in range(0,len(Y)):
        node_attr['triangle'][i] = int(X[i][0])
        node_attr['hexagon'][i] = int(Y[i][0])
      x = node_attr.to_numpy()
      x=torch.from_numpy(x)
      y=np.array([1])
      y=torch.from_numpy(y) 
      adj_t = torch.tensor(A)
      edge_index = adj_t.nonzero().t().contiguous()
      data = Data(x=x, edge_index=edge_index,y=y)
      datasets1.append(data)
      continue
  else:
    break
   
print(count)

1899
1892
1897
1895
1896
1894
19
1893
1885
1891
1889
1887
1888
1883
1890
1884
1886
189
1879
188
1880
1875
1874
1881
1877
1882
1876
1878
1870
187
1871
1867
1865
1866
1868
1873
1869
1872
1858
1862
1860
1864
1857
1856
1863
1859
1861
186
1853
1854
1850
185
1849
1855
1847
1852
1848
1851
1841
184
1845
1842
1838
1843
1839
1840
1846
1844
1831
1835
1832
1829
1830
1834
1833
1836
1837
183
1828
1825
182
1822
1824
1821
1823
1827
1826
1820
1811
1819
1812
1814
1817
1815
1816
1818
1810
1813
181
1809
1807
1801
1803
1806
1805
1802
1808
1804
1798
1800
1796
1795
1799
180
18
1793
1794
1797
179
1790
1786
1791
1784
1792
1785
1789
1788
1787
1782
1779
1777
1783
1776
1778
1775
1781
1780
178
1768
1770
1769
1772
1774
177
1766
1771
1773
1767
1765
1762
1758
1757
1759
1763
1764
176
1761
1760
1752
1749
1750
175
1748
1756
1755
1754
1751
1753
1747
1746
1745
1740
1743
1741
1744
1742
1739
174
1738
1733
1736
1737
1730
173
1732
1731
1734
1735
1727
1729
1722
1725
1721
1728
1723
1720
1726
1724
1715
1712
1718
1713
1717
1711
1

In [55]:
dataset3 = datasets1[:406]

In [23]:
torch.save(dataset3,'hexagon_406_1.pt')

In [137]:
data1 = torch.load('hexagon_406_1.pt')
data2 = torch.load('hexagon_407_0.pt')

In [138]:
dataset = data2 + data1

In [139]:
dataset

[Data(x=[93, 6], edge_index=[2, 176], y=[1]),
 Data(x=[95, 6], edge_index=[2, 160], y=[1]),
 Data(x=[24, 6], edge_index=[2, 58], y=[1]),
 Data(x=[9, 6], edge_index=[2, 8], y=[1]),
 Data(x=[61, 6], edge_index=[2, 32], y=[1]),
 Data(x=[21, 6], edge_index=[2, 78], y=[1]),
 Data(x=[18, 6], edge_index=[2, 28], y=[1]),
 Data(x=[70, 6], edge_index=[2, 280], y=[1]),
 Data(x=[52, 6], edge_index=[2, 456], y=[1]),
 Data(x=[5, 6], edge_index=[2, 10], y=[1]),
 Data(x=[97, 6], edge_index=[2, 2080], y=[1]),
 Data(x=[24, 6], edge_index=[2, 62], y=[1]),
 Data(x=[8, 6], edge_index=[2, 12], y=[1]),
 Data(x=[97, 6], edge_index=[2, 228], y=[1]),
 Data(x=[55, 6], edge_index=[2, 230], y=[1]),
 Data(x=[58, 6], edge_index=[2, 116], y=[1]),
 Data(x=[90, 6], edge_index=[2, 120], y=[1]),
 Data(x=[89, 6], edge_index=[2, 52], y=[1]),
 Data(x=[98, 6], edge_index=[2, 288], y=[1]),
 Data(x=[11, 6], edge_index=[2, 20], y=[1]),
 Data(x=[48, 6], edge_index=[2, 528], y=[1]),
 Data(x=[48, 6], edge_index=[2, 50], y=[1]),
 D

In [128]:
torch.save(dataset,'hexagon_812_final.pt')